# Neural Networks from Scratch

Building foundational understanding of neural networks by implementing everything manually.

**Goal**: Understand every computation that happens during forward and backward passes.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(42)

---
## Part 1: Single Neuron (Perceptron / Logistic Regression)

The simplest neural network: one neuron that learns to classify.

**Your notes here:**


In [ ]:
# Generate simple 2D classification data
# Two clusters of points we want to separate

def generate_data(n_samples=100):
    # Class 0: centered at (-1, -1)
    X0 = np.random.randn(n_samples // 2, 2) * 0.5 + np.array([-1, -1])
    # Class 1: centered at (1, 1)
    X1 = np.random.randn(n_samples // 2, 2) * 0.5 + np.array([1, 1])
    
    X = np.vstack([X0, X1])
    y = np.array([0] * (n_samples // 2) + [1] * (n_samples // 2))
    
    return X, y

X, y = generate_data(200)

# Visualize
plt.figure(figsize=(8, 6))
plt.scatter(X[y == 0, 0], X[y == 0, 1], c='blue', label='Class 0')
plt.scatter(X[y == 1, 0], X[y == 1, 1], c='red', label='Class 1')
plt.xlabel('x1')
plt.ylabel('x2')
plt.legend()
plt.title('Binary Classification Data')
plt.show()

### Sigmoid Activation

Squashes any real number to range (0, 1) - interpretable as probability.

**Your notes here:**


In [ ]:
def sigmoid(z):
    """Sigmoid activation: maps R -> (0, 1)"""
    return 1 / (1 + np.exp(-z))

def sigmoid_derivative(z):
    """Derivative of sigmoid: sigmoid(z) * (1 - sigmoid(z))"""
    s = sigmoid(z)
    return s * (1 - s)

# Visualize sigmoid and its derivative
z = np.linspace(-6, 6, 100)
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(z, sigmoid(z))
axes[0].axhline(y=0.5, color='r', linestyle='--', alpha=0.5)
axes[0].axvline(x=0, color='r', linestyle='--', alpha=0.5)
axes[0].set_title('Sigmoid Function')
axes[0].set_xlabel('z')
axes[0].set_ylabel('sigmoid(z)')
axes[0].grid(True)

axes[1].plot(z, sigmoid_derivative(z))
axes[1].set_title('Sigmoid Derivative (for backprop)')
axes[1].set_xlabel('z')
axes[1].set_ylabel("sigmoid'(z)")
axes[1].grid(True)

plt.tight_layout()
plt.show()

### Single Neuron: Forward Pass

```
z = w1*x1 + w2*x2 + b    (linear combination)
a = sigmoid(z)            (activation)
```

**Your notes here:**


In [ ]:
class SingleNeuron:
    def __init__(self, n_features):
        # Initialize weights randomly, bias to zero
        self.w = np.random.randn(n_features) * 0.01
        self.b = 0.0
        
    def forward(self, X):
        """
        Forward pass:
        X: (n_samples, n_features)
        Returns: predictions (n_samples,)
        """
        # Linear combination: z = X @ w + b
        self.z = np.dot(X, self.w) + self.b
        # Activation
        self.a = sigmoid(self.z)
        return self.a
    
    def compute_loss(self, y_true, y_pred):
        """
        Binary Cross-Entropy Loss:
        L = -[y*log(p) + (1-y)*log(1-p)]
        
        Why this loss? It's derived from maximum likelihood estimation.
        """
        # Clip to avoid log(0)
        eps = 1e-15
        y_pred = np.clip(y_pred, eps, 1 - eps)
        
        loss = -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
        return loss
    
    def backward(self, X, y_true):
        """
        Backward pass - compute gradients manually.
        
        Chain rule:
        dL/dw = dL/da * da/dz * dz/dw
        
        For BCE + sigmoid, this simplifies beautifully:
        dL/dz = a - y  (prediction error!)
        """
        n_samples = X.shape[0]
        
        # Gradient of loss w.r.t. pre-activation (this is the key insight)
        dz = self.a - y_true  # (n_samples,)
        
        # Gradient of loss w.r.t. weights
        # dL/dw = X^T @ dz / n_samples
        self.dw = np.dot(X.T, dz) / n_samples
        
        # Gradient of loss w.r.t. bias
        self.db = np.mean(dz)
        
        return self.dw, self.db
    
    def update(self, lr=0.1):
        """Gradient descent update"""
        self.w -= lr * self.dw
        self.b -= lr * self.db
    
    def predict(self, X):
        """Binary prediction (threshold at 0.5)"""
        return (self.forward(X) >= 0.5).astype(int)

### Training the Single Neuron

**Your notes here:**


In [ ]:
# Train the single neuron
neuron = SingleNeuron(n_features=2)
losses = []

for epoch in range(100):
    # Forward
    y_pred = neuron.forward(X)
    loss = neuron.compute_loss(y, y_pred)
    losses.append(loss)
    
    # Backward
    neuron.backward(X, y)
    
    # Update
    neuron.update(lr=1.0)
    
    if epoch % 20 == 0:
        acc = np.mean(neuron.predict(X) == y)
        print(f"Epoch {epoch}: Loss={loss:.4f}, Accuracy={acc:.2%}")

# Plot loss curve
plt.figure(figsize=(8, 4))
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.grid(True)
plt.show()

In [ ]:
# Visualize the decision boundary
def plot_decision_boundary(model, X, y):
    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100),
                         np.linspace(y_min, y_max, 100))
    
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    plt.figure(figsize=(8, 6))
    plt.contourf(xx, yy, Z, alpha=0.3, cmap='RdBu')
    plt.scatter(X[y == 0, 0], X[y == 0, 1], c='blue', label='Class 0')
    plt.scatter(X[y == 1, 0], X[y == 1, 1], c='red', label='Class 1')
    plt.xlabel('x1')
    plt.ylabel('x2')
    plt.legend()
    plt.title('Decision Boundary (Single Neuron)')
    plt.show()

plot_decision_boundary(neuron, X, y)
print(f"Learned weights: w={neuron.w}, b={neuron.b:.4f}")

---
## Part 2: Two-Layer MLP with Manual Backprop

Now we add a hidden layer. This lets us learn non-linear decision boundaries.

```
Input (2) -> Hidden (4) -> Output (1)
```

**Your notes here:**


In [ ]:
# Generate XOR-like data (not linearly separable!)
def generate_xor_data(n_samples=200):
    X = np.random.randn(n_samples, 2)
    # XOR: label is 1 if signs of x1 and x2 are different
    y = ((X[:, 0] * X[:, 1]) < 0).astype(int)
    return X, y

X_xor, y_xor = generate_xor_data(300)

plt.figure(figsize=(8, 6))
plt.scatter(X_xor[y_xor == 0, 0], X_xor[y_xor == 0, 1], c='blue', label='Class 0')
plt.scatter(X_xor[y_xor == 1, 0], X_xor[y_xor == 1, 1], c='red', label='Class 1')
plt.xlabel('x1')
plt.ylabel('x2')
plt.axhline(y=0, color='gray', linestyle='--', alpha=0.3)
plt.axvline(x=0, color='gray', linestyle='--', alpha=0.3)
plt.legend()
plt.title('XOR Data (NOT linearly separable)')
plt.show()

### The Math of Backpropagation

For a 2-layer network:

**Forward:**
```
z1 = X @ W1 + b1
a1 = relu(z1)           # hidden activation
z2 = a1 @ W2 + b2
a2 = sigmoid(z2)        # output
```

**Backward (chain rule):**
```
dz2 = a2 - y                    # gradient at output
dW2 = a1.T @ dz2 / n
db2 = mean(dz2)

da1 = dz2 @ W2.T                # backprop through weights
dz1 = da1 * relu_derivative(z1) # backprop through activation
dW1 = X.T @ dz1 / n
db1 = mean(dz1)
```

**Your notes here:**


In [ ]:
def relu(z):
    """ReLU activation: max(0, z)"""
    return np.maximum(0, z)

def relu_derivative(z):
    """Derivative of ReLU: 1 if z > 0, else 0"""
    return (z > 0).astype(float)

class TwoLayerMLP:
    def __init__(self, n_input, n_hidden, n_output):
        # Xavier initialization for weights
        self.W1 = np.random.randn(n_input, n_hidden) * np.sqrt(2.0 / n_input)
        self.b1 = np.zeros(n_hidden)
        self.W2 = np.random.randn(n_hidden, n_output) * np.sqrt(2.0 / n_hidden)
        self.b2 = np.zeros(n_output)
        
    def forward(self, X):
        """
        Forward pass - save intermediates for backprop.
        X: (batch_size, n_input)
        """
        # Layer 1
        self.z1 = np.dot(X, self.W1) + self.b1  # (batch, n_hidden)
        self.a1 = relu(self.z1)                  # (batch, n_hidden)
        
        # Layer 2 (output)
        self.z2 = np.dot(self.a1, self.W2) + self.b2  # (batch, n_output)
        self.a2 = sigmoid(self.z2)                     # (batch, n_output)
        
        return self.a2.squeeze()
    
    def backward(self, X, y_true):
        """
        Backward pass - compute all gradients via chain rule.
        This is the heart of understanding neural networks!
        """
        n_samples = X.shape[0]
        y_true = y_true.reshape(-1, 1)  # (batch, 1)
        
        # ===== OUTPUT LAYER (Layer 2) =====
        # dL/dz2 = a2 - y (same as single neuron)
        dz2 = self.a2 - y_true  # (batch, 1)
        
        # dL/dW2 = a1^T @ dz2
        self.dW2 = np.dot(self.a1.T, dz2) / n_samples  # (n_hidden, 1)
        self.db2 = np.mean(dz2, axis=0)  # (1,)
        
        # ===== HIDDEN LAYER (Layer 1) =====
        # Backprop through W2: da1 = dz2 @ W2^T
        da1 = np.dot(dz2, self.W2.T)  # (batch, n_hidden)
        
        # Backprop through ReLU: dz1 = da1 * relu'(z1)
        dz1 = da1 * relu_derivative(self.z1)  # (batch, n_hidden)
        
        # dL/dW1 = X^T @ dz1
        self.dW1 = np.dot(X.T, dz1) / n_samples  # (n_input, n_hidden)
        self.db1 = np.mean(dz1, axis=0)  # (n_hidden,)
    
    def update(self, lr):
        """Gradient descent update"""
        self.W1 -= lr * self.dW1
        self.b1 -= lr * self.db1
        self.W2 -= lr * self.dW2
        self.b2 -= lr * self.db2
    
    def predict(self, X):
        return (self.forward(X) >= 0.5).astype(int)
    
    def compute_loss(self, y_true, y_pred):
        eps = 1e-15
        y_pred = np.clip(y_pred, eps, 1 - eps)
        return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

In [ ]:
# Train on XOR data
mlp = TwoLayerMLP(n_input=2, n_hidden=8, n_output=1)
losses = []

for epoch in range(1000):
    # Forward
    y_pred = mlp.forward(X_xor)
    loss = mlp.compute_loss(y_xor, y_pred)
    losses.append(loss)
    
    # Backward
    mlp.backward(X_xor, y_xor)
    
    # Update
    mlp.update(lr=1.0)
    
    if epoch % 200 == 0:
        acc = np.mean(mlp.predict(X_xor) == y_xor)
        print(f"Epoch {epoch}: Loss={loss:.4f}, Accuracy={acc:.2%}")

# Plot
plt.figure(figsize=(8, 4))
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('MLP Training on XOR')
plt.grid(True)
plt.show()

In [ ]:
# Visualize the non-linear decision boundary
plot_decision_boundary(mlp, X_xor, y_xor)
print(f"Final accuracy: {np.mean(mlp.predict(X_xor) == y_xor):.2%}")

---
## Part 3: Activation Functions Deep Dive

Different activations have different properties. Understanding these is crucial.

**Your notes here:**


In [ ]:
# All common activation functions

def sigmoid(z):
    return 1 / (1 + np.exp(-np.clip(z, -500, 500)))

def tanh(z):
    return np.tanh(z)

def relu(z):
    return np.maximum(0, z)

def leaky_relu(z, alpha=0.01):
    return np.where(z > 0, z, alpha * z)

def gelu(z):
    """Gaussian Error Linear Unit - used in transformers (BERT, GPT)"""
    return 0.5 * z * (1 + np.tanh(np.sqrt(2 / np.pi) * (z + 0.044715 * z**3)))

def swish(z):
    """Self-gated activation: z * sigmoid(z)"""
    return z * sigmoid(z)

In [ ]:
# Visualize all activations
z = np.linspace(-4, 4, 200)

fig, axes = plt.subplots(2, 3, figsize=(14, 8))
activations = [
    ('Sigmoid', sigmoid),
    ('Tanh', tanh),
    ('ReLU', relu),
    ('Leaky ReLU', leaky_relu),
    ('GELU', gelu),
    ('Swish', swish)
]

for ax, (name, func) in zip(axes.flat, activations):
    ax.plot(z, func(z), linewidth=2)
    ax.axhline(y=0, color='gray', linestyle='--', alpha=0.3)
    ax.axvline(x=0, color='gray', linestyle='--', alpha=0.3)
    ax.set_title(name)
    ax.set_xlabel('z')
    ax.grid(True, alpha=0.3)
    ax.set_xlim(-4, 4)

plt.tight_layout()
plt.show()

### Why Different Activations?

| Activation | Pros | Cons | Used In |
|------------|------|------|--------|
| Sigmoid | Bounded (0,1), interpretable as prob | Vanishing gradient, not zero-centered | Output layer (binary) |
| Tanh | Zero-centered, bounded (-1,1) | Vanishing gradient | Old RNNs |
| ReLU | Fast, no vanishing gradient for z>0 | Dead neurons (z<0 never recovers) | Most CNNs |
| Leaky ReLU | No dead neurons | Extra hyperparameter | CNNs |
| GELU | Smooth, probabilistic interpretation | Slower to compute | Transformers (BERT, GPT) |
| Swish | Self-gated, smooth | Slower | Some modern nets |

**Your notes here:**


### Vanishing Gradient Problem

**Your notes here:**


In [ ]:
# Demonstrate vanishing gradients with sigmoid
def sigmoid_derivative(z):
    s = sigmoid(z)
    return s * (1 - s)

# Simulate deep network: gradient gets multiplied at each layer
# If derivative < 1, gradient shrinks exponentially

z = np.linspace(-6, 6, 100)

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Sigmoid derivative is always < 0.25!
axes[0].plot(z, sigmoid_derivative(z))
axes[0].axhline(y=0.25, color='r', linestyle='--', label='max = 0.25')
axes[0].set_title('Sigmoid Derivative')
axes[0].set_xlabel('z')
axes[0].legend()
axes[0].grid(True)

# After n layers, gradient is multiplied by (derivative)^n
n_layers = np.arange(1, 20)
gradient_sigmoid = 0.25 ** n_layers  # worst case for sigmoid
gradient_relu = 1.0 ** n_layers  # ReLU maintains gradient (when active)

axes[1].semilogy(n_layers, gradient_sigmoid, 'b-', label='Sigmoid (worst case)')
axes[1].semilogy(n_layers, gradient_relu, 'g-', label='ReLU (active)')
axes[1].set_title('Gradient Magnitude vs Network Depth')
axes[1].set_xlabel('Number of Layers')
axes[1].set_ylabel('Gradient Scale (log)')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

print(f"After 10 sigmoid layers, gradient is ~{0.25**10:.2e} of original")
print("This is why deep sigmoid networks don't train well!")

---
## Part 4: Weight Initialization

Proper initialization is critical for training deep networks.

**Your notes here:**


In [ ]:
def xavier_init(fan_in, fan_out):
    """
    Xavier/Glorot initialization (for tanh/sigmoid).
    Variance = 2 / (fan_in + fan_out)
    
    Goal: Keep variance of activations stable across layers.
    """
    std = np.sqrt(2.0 / (fan_in + fan_out))
    return np.random.randn(fan_in, fan_out) * std

def kaiming_init(fan_in, fan_out):
    """
    Kaiming/He initialization (for ReLU).
    Variance = 2 / fan_in
    
    Accounts for ReLU zeroing out half the activations.
    """
    std = np.sqrt(2.0 / fan_in)
    return np.random.randn(fan_in, fan_out) * std

def bad_init(fan_in, fan_out):
    """Too large - will cause exploding activations"""
    return np.random.randn(fan_in, fan_out) * 1.0

def tiny_init(fan_in, fan_out):
    """Too small - activations will vanish"""
    return np.random.randn(fan_in, fan_out) * 0.001

In [ ]:
# Demonstrate impact of initialization
def simulate_forward_pass(init_func, n_layers=10, layer_size=256):
    """Simulate forward pass through many layers, track activation stats"""
    x = np.random.randn(32, layer_size)  # batch of 32
    activation_means = []
    activation_stds = []
    
    for i in range(n_layers):
        W = init_func(layer_size, layer_size)
        x = relu(np.dot(x, W))
        activation_means.append(np.mean(np.abs(x)))
        activation_stds.append(np.std(x))
    
    return activation_means, activation_stds

# Compare different initializations
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

for name, init_func in [('Xavier', xavier_init), ('Kaiming', kaiming_init), 
                         ('Too Large', bad_init), ('Too Small', tiny_init)]:
    means, stds = simulate_forward_pass(init_func)
    axes[0].plot(means, label=name)
    axes[1].plot(stds, label=name)

axes[0].set_title('Mean Activation Magnitude per Layer')
axes[0].set_xlabel('Layer')
axes[0].set_ylabel('Mean |activation|')
axes[0].legend()
axes[0].set_yscale('log')
axes[0].grid(True)

axes[1].set_title('Activation Std Dev per Layer')
axes[1].set_xlabel('Layer')
axes[1].set_ylabel('Std Dev')
axes[1].legend()
axes[1].set_yscale('log')
axes[1].grid(True)

plt.tight_layout()
plt.show()

print("Notice: Kaiming keeps activations stable with ReLU!")
print("Too large -> explodes, Too small -> vanishes")

---
## Part 5: Batch Normalization vs Layer Normalization

Normalization techniques stabilize training by controlling activation distributions.

**Your notes here:**


In [ ]:
class BatchNorm:
    """
    Batch Normalization: normalize across the BATCH dimension.
    
    For input shape (batch_size, features):
    - Compute mean/var across batch_size for each feature
    - Each feature has its own mean/var
    
    Used in: CNNs, image models
    Problem: Depends on batch size, doesn't work well for small batches or RNNs
    """
    def __init__(self, n_features, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.momentum = momentum
        
        # Learnable parameters
        self.gamma = np.ones(n_features)   # scale
        self.beta = np.zeros(n_features)   # shift
        
        # Running statistics for inference
        self.running_mean = np.zeros(n_features)
        self.running_var = np.ones(n_features)
        
    def forward(self, x, training=True):
        """
        x: (batch_size, n_features)
        """
        if training:
            # Compute batch statistics
            mean = np.mean(x, axis=0)  # mean over batch -> (n_features,)
            var = np.var(x, axis=0)    # var over batch -> (n_features,)
            
            # Update running statistics
            self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * mean
            self.running_var = (1 - self.momentum) * self.running_var + self.momentum * var
        else:
            mean = self.running_mean
            var = self.running_var
        
        # Normalize
        x_norm = (x - mean) / np.sqrt(var + self.eps)
        
        # Scale and shift
        out = self.gamma * x_norm + self.beta
        
        return out

In [ ]:
class LayerNorm:
    """
    Layer Normalization: normalize across the FEATURE dimension.
    
    For input shape (batch_size, features):
    - Compute mean/var across features for each sample
    - Each sample has its own mean/var
    
    Used in: Transformers, RNNs
    Advantage: Independent of batch size, works for any sequence length
    """
    def __init__(self, n_features, eps=1e-5):
        self.eps = eps
        
        # Learnable parameters
        self.gamma = np.ones(n_features)   # scale
        self.beta = np.zeros(n_features)   # shift
        
    def forward(self, x):
        """
        x: (batch_size, n_features) or (batch_size, seq_len, n_features)
        """
        # Compute statistics across last dimension (features)
        mean = np.mean(x, axis=-1, keepdims=True)  # (batch,) or (batch, seq, 1)
        var = np.var(x, axis=-1, keepdims=True)
        
        # Normalize
        x_norm = (x - mean) / np.sqrt(var + self.eps)
        
        # Scale and shift
        out = self.gamma * x_norm + self.beta
        
        return out

In [ ]:
# Visualize the difference
batch_size = 4
n_features = 8

# Create sample data with different scales per feature
x = np.random.randn(batch_size, n_features)
x[:, 0] *= 10  # feature 0 has large scale
x[:, 1] *= 0.1  # feature 1 has small scale

bn = BatchNorm(n_features)
ln = LayerNorm(n_features)

x_bn = bn.forward(x)
x_ln = ln.forward(x)

fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# Original
im0 = axes[0].imshow(x, aspect='auto', cmap='RdBu', vmin=-3, vmax=3)
axes[0].set_title('Original')
axes[0].set_xlabel('Features')
axes[0].set_ylabel('Batch Samples')
plt.colorbar(im0, ax=axes[0])

# BatchNorm
im1 = axes[1].imshow(x_bn, aspect='auto', cmap='RdBu', vmin=-3, vmax=3)
axes[1].set_title('After BatchNorm\n(normalize per COLUMN)')
axes[1].set_xlabel('Features')
axes[1].set_ylabel('Batch Samples')
plt.colorbar(im1, ax=axes[1])

# LayerNorm
im2 = axes[2].imshow(x_ln, aspect='auto', cmap='RdBu', vmin=-3, vmax=3)
axes[2].set_title('After LayerNorm\n(normalize per ROW)')
axes[2].set_xlabel('Features')
axes[2].set_ylabel('Batch Samples')
plt.colorbar(im2, ax=axes[2])
plt.tight_layout()
plt.show()

print("BatchNorm: Each FEATURE (column) has mean~0, var~1")
print(f"  Feature means: {x_bn.mean(axis=0).round(2)}")
print(f"\nLayerNorm: Each SAMPLE (row) has mean~0, var~1")
print(f"  Sample means: {x_ln.mean(axis=1).round(2)}")

### When to use which?

| Normalization | Normalize Across | Best For | Why |
|--------------|------------------|----------|-----|
| BatchNorm | Batch (samples) | CNNs, fixed input size | Features should have consistent scale |
| LayerNorm | Features | Transformers, RNNs | Batch-independent, variable sequence length |

**Your notes here:**


---
## Summary: Key Takeaways

1. **Single neuron** = linear decision boundary, can only solve linearly separable problems
2. **Hidden layers** = can learn non-linear boundaries (universal approximation)
3. **Backpropagation** = chain rule applied systematically through the network
4. **Activation functions** matter: ReLU for depth, GELU for transformers
5. **Initialization** must preserve signal variance (Xavier/Kaiming)
6. **Normalization** stabilizes training (BatchNorm for CNNs, LayerNorm for transformers)

**Your notes here:**
